<br>
<br>

![iteso](https://upload.wikimedia.org/wikipedia/en/5/5f/Western_Institute_of_Technology_and_Higher_Education_logo.png)

### InstitutoTecnológico y de Estudios Superiores de Occidente ###
### Maestría Ciencia de Datos  ###
### Aprendizaje Profundo ###
### Actividad 3 ###

<br>
<br>

* * *

Estudiante: Daniel Nuño <br>
Profesor: Dr. Francisco Cervantes <br>
Fecha entrega: February 12, 2023 <br>

* * *

<br>
<br>

Creo que el uso de imagenes segmentas da una ligera ventaja ya que el fondo es negro, resalta la hoja, y la posible enfermedad. El primer paso es importar las imagenes:

- recusivamente importar las imagenes de la carpeta *segmented*.
    - primero intentar con escala de grises y despues a color.
- cambiar el tamaño para que sea más manejable.
    - primero intentar con el tamaño original, despues reduciendo el tamaño, después con Singular Value Decomposition para comprimir manteniendo las caracteristicas.
- rotar la matrix 90 grados 3 veces con la intención de generar 3 veces más datos.
- aplanarlos para tener una matrix de cuatro observaciones.
- usar el nombre de la carpeta para obtener las clasificaciones: tipo de hoja y estado de salud.

La matriz X esperada es:
- 56,304 x 4 = 225,216 filas
- dependiendo de la forma de importar de las imagenes X cantidad de columnas

La matriz Y esperada es:
- misma cantidad de filas que X
- 14 columnas por la especie más 26 columnas por enfermedades (o una columna para determinar si esta sana o no) = 40 (o 15)

In [143]:
from PIL import Image
import numpy as np
import os

main_folder_path = 'C:/Users/nuno/Desktop/deep-learning-data/activity3/' + 'segmented/'

In [208]:
counter = 0
metadata_img = []
arr = np.zeros((54306*4, 32*32*1)) #32*32*1 img size  and 4 rotations

for dirpath, dirnames, filenames in os.walk(main_folder_path):
    for filename in filenames:
        dirs_paths_state = [os.path.join(dirpath, filename),
                            str(os.path.basename(dirpath)).split('___')[0],
                            str(os.path.basename(dirpath)).split('___')[1]
                            ]
        img = Image.open(os.path.join(dirpath, filename))
        img = img.convert('L')
        img = img.resize((32, 32))

        for times in range(4):
            metadata_img.append(dirs_paths_state)
            arr[times + (counter*4)] = np.ravel(np.array(img))
            img = img.rotate(angle=90)
        
        counter += 1

img.close()

In [209]:
len(metadata_img)

217224

In [210]:
54306*4

217224

In [191]:
arr[-1].min()

0.0

In [183]:
(counter-1)*4

217220

In [184]:
arr.shape

(225216, 1024)

In [186]:
56304*4

225216